In [1]:
# Run this cell if you are using Google Colab or any other notebook cloud service
# If you are using a local notebook server, you can skip this cell
!git clone https://github.com/isa-group/saas-pricing

Cloning into 'saas-pricing'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 57 (delta 19), reused 45 (delta 10), pack-reused 0
Receiving objects: 100% (57/57), 435.64 KiB | 4.03 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [2]:
import pandas as pd
import plotly.graph_objects as go

In [3]:
YEARS_STUDIED = ["2019", "2020", "2022", "2023"]
COLORS = ["orange", "yellow", "green", "blue"]

In [4]:
PRICING_DATA = pd.read_excel("saas-pricing/datasets/pricingData.xlsx")

print(PRICING_DATA.head())

     saasName  year  informationFeatures  integrationFeatures  toolFeatures  \
0    Clockify  2023                   10                    4            23   
1      Github  2023                    3                    5            19   
2     Postman  2023                    4                    4            17   
3  Sonarcloud  2023                    0                    1             4   
4     Navetor  2023                    0                    0             7   

   automationFeatures  managementFeatures  numberOfFeatures  numberOfAddOns  
0                   4                  26                67               0  
1                   8                  10                45               5  
2                   1                   8                34               4  
3                   1                   2                 8               0  
4                   1                   0                 8               1  


In [14]:
# DEFINITION OF USEFUL FUNCTIONS

def get_saas_names():
  return tuple(set(PRICING_DATA['saasName'].to_list()))

def create_chart_data(objective_attribute):
  """
  Builds the data needed to create a dumbell chart to see the evolution of a field between 2019 and 2023
  """
  chart_data = {"line_x": [], "line_y": [], "2019": [], "2020": [], "2022": [], "2023": []}

  saasNames = get_saas_names()

  for saas in saasNames:

      max_value_of_saas = float("-inf")
      min_value_of_saas = float("+inf")

      for year in YEARS_STUDIED:

          try:
              saas_year_data = PRICING_DATA.loc[(PRICING_DATA.year == int(year)) & (PRICING_DATA.saasName == saas)][objective_attribute].values[0]
              if saas_year_data > max_value_of_saas:
                  max_value_of_saas = saas_year_data

              if saas_year_data < min_value_of_saas:
                  min_value_of_saas = saas_year_data

          except IndexError:
              saas_year_data = None

          chart_data[year].extend([saas_year_data])

      chart_data["line_x"].extend([
          max_value_of_saas,
          min_value_of_saas,
          None
      ])
      chart_data["line_y"].extend([saas, saas, None])

  return chart_data

def plot_chart(chart_data, title, x_axis_name):
  """
  Plots a dumbell char to see the evolution of a field between 2019 and 2023
  """

  data = data=[
          go.Scatter(
              x=chart_data["line_x"],
              y=chart_data["line_y"],
              mode="lines",
              showlegend=False,
              marker=dict(
                  color="grey"
              )
            )
          ]

  for i in range(len(YEARS_STUDIED)):

    year = YEARS_STUDIED[i]
    color = COLORS[i]

    data.append(
        go.Scatter(
              x=chart_data[year],
              y=get_saas_names(),
              mode="markers",
              name=year,
              marker=dict(
                  color=color,
                  size=10
              )

          )
    )

  fig = go.Figure(
      data=data
  )

  fig.update_layout(
      title=title,
      height=1000,
      legend_itemclick=False,
      xaxis_title=x_axis_name
  )

  fig.show()

def create_spaces(number_of_spaces):

  final_string = ""

  for i in range(number_of_spaces):
    final_string += " "

  return final_string

def build_ascii_table (table, **k):
    header = k.get('header', [])
    align = k.get('align', 'left')
    border = k.get('border', False)

    widths = []
    for i in range(max(map(len, table))): widths.append(max(max(map(len, [row[i] for row in table if len(row) > i])), len(header[i]) if len(header) > i else 0))

    printable = []

    if border:
        printrow = []
        for i in range(max(map(len, table))):
            if i > 0 and i < max(map(len, table)) - 1: printrow.append('─' * (widths[i] + 2))
            else: printrow.append('─' * (widths[i] + 1))
        printable.append('┌─' + '┬'.join(printrow) + '─┐')

    # header formatting
    if len(header) > 0:
        printrow = []
        for i in range(len(header)):
            assert header[i]
            if align == 'center': printrow.append(header[i].center(widths[i]))
            elif align == 'left': printrow.append(header[i].ljust(widths[i]))
            elif align == 'right': printrow.append(header[i].rjust(widths[i]))

        if border: printable.append('│ ' + ' │ '.join(printrow) + ' │')
        else: printable.append(' │ '.join(printrow))

        printrow = []
        for i in range(len(header)):
            if i > 0 and i < len(header) - 1: printrow.append('─' * (widths[i] + 2))
            else: printrow.append('─' * (widths[i] + 1))

        if border: printable.append('├─' + '┼'.join(printrow) + '─┤')
        else: printable.append('┼'.join(printrow))

    # table formatting
    for row in table:
        printrow = []
        for i in range(len(widths) - len(row)):
            row.append('')
        for i in range(len(row)):
            if align == 'center': printrow.append(row[i].center(widths[i]))
            elif align == 'left': printrow.append(row[i].ljust(widths[i]))
            elif align == 'right': printrow.append(row[i].rjust(widths[i]))

        if border: printable.append('│ ' + ' │ '.join(printrow) + ' │')
        else: printable.append(' │ '.join(printrow))

    if border:
        printrow = []
        for i in range(max(map(len, table))):
            if i > 0 and i < max(map(len, table)) - 1: printrow.append('─' * (widths[i] + 2))
            else: printrow.append('─' * (widths[i] + 1))
        printable.append('└─' + '┴'.join(printrow) + '─┘')

    return '\n'.join(printable)

In [15]:
plot_chart(create_chart_data("numberOfFeatures"), "Evolution of features in SaaS pricings between 2019 and 2023", "Number of Features")

In [16]:
plot_chart(create_chart_data("numberOfAddOns"), "Evolution of Add-Ons in SaaS pricing between 2019 and 2023", "Number of Add-Ons")

In [8]:
# CALCULATION OF AVERAGE PERCENTAGE VARIATION IN THE NUMBER OF FEATURES BETWEEN 2019 AND 2023

saasNames = get_saas_names()

sum_of_variations = 0
sum_of_first_year_values = 0

headers = ["Saas Name", "Percentage of variation", "First entry's year", "Last entry's year"]
rows = []

for saas in saasNames:

    saas_entries = PRICING_DATA.loc[PRICING_DATA['saasName'] == saas].sort_values(by='year')

    first_year_entry = saas_entries.iloc[0]
    last_year_entry = saas_entries.iloc[-1]

    saas_feature_variation = last_year_entry['numberOfFeatures'] - first_year_entry['numberOfFeatures']

    if saas_feature_variation != 0:
      sum_of_variations += saas_feature_variation
      sum_of_first_year_values += first_year_entry['numberOfFeatures']

      rows.append((saas,f"{int(saas_feature_variation/first_year_entry['numberOfFeatures']*100)}%",f"{first_year_entry['year']}", f"{last_year_entry['year']}"))

ascii_table = build_ascii_table(rows, header=headers, align='center', border=True)

print(ascii_table)

average_variation = sum_of_variations / sum_of_first_year_values * 100

print(f"\nThere has been, on average, an increase of {int(average_variation)}% in the number of features included in pricing of the SaaS studied")

┌────────────┬─────────────────────────┬────────────────────┬───────────────────┐
│ Saas Name  │ Percentage of variation │ First entry's year │ Last entry's year │
├────────────┼─────────────────────────┼────────────────────┼───────────────────┤
│   Monday   │           82%           │        2019        │        2023       │
│   Wrike    │           92%           │        2019        │        2023       │
│  Overleaf  │           40%           │        2019        │        2023       │
│  Postman   │           161%          │        2020        │        2023       │
│  Clockify  │           86%           │        2022        │        2023       │
│    Jira    │           89%           │        2020        │        2023       │
│ Sonarcloud │           -33%          │        2019        │        2023       │
│   Canva    │           180%          │        2019        │        2023       │
│  Navetor   │           -33%          │        2020        │        2023       │
│ Salescloud │  